In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.cm as cm
import seaborn as sns
import sys
import os
import shutil

'/Users/nguyenthiminhhien/CEL_'

In [20]:
source="Simple Data.zip"
shutil.unpack_archive(source,'./','zip')

In [10]:
Sales_Test="sales_test.csv"
Canceled_Test="canceled_test.csv"
Sales_Df=pd.read_csv(Sales_Test)
Canceled_Df=pd.read_csv(Canceled_Test)

In [41]:
Sales_Df.head(10)

,Order No,Order Date,Line No,Ship To Address No,Item Code,Order Qty,Quantity Shipped
0,528758,2017-01-03,1,1358538.0,111931,70.0,70.0
1,528791,2017-01-03,1,1254798.0,1029071,10.0,10.0
2,528791,2017-01-03,2,1254798.0,1033341,10.0,10.0
3,528791,2017-01-03,3,1254798.0,1040827,5.0,5.0
4,528791,2017-01-03,4,1254798.0,10106111,10.0,10.0
5,528791,2017-01-03,5,1254798.0,10105916,15.0,15.0
6,528791,2017-01-03,6,1254798.0,10106230,15.0,15.0
7,528791,2017-01-03,7,1254798.0,10106229,15.0,15.0
8,528791,2017-01-03,8,1254798.0,1040726,15.0,15.0
9,528791,2017-01-03,9,1254798.0,10098739,550.0,550.0


In [40]:
Sales_Df.describe()

,Order No,Line No,Ship To Address No,Item Code,Order Qty,Quantity Shipped
count,28315.000000,28315.000000,2.831500e+04,2.831500e+04,28315.000000,28315.000000
mean,536330.828289,3.548508,8.571873e+06,4.897559e+06,6.471305,6.471305
std,4339.727923,3.388066,1.563677e+07,4.564069e+06,36.203671,36.203671
min,528758.000000,1.000000,1.254612e+06,1.104410e+05,1.000000,1.000000
25%,532522.000000,1.000000,1.255615e+06,1.040725e+06,1.000000,1.000000
50%,536359.000000,2.000000,1.740542e+06,1.041106e+06,1.000000,1.000000
75%,540127.500000,5.000000,2.247534e+06,1.013370e+07,2.000000,2.000000
max,543766.000000,47.000000,4.382536e+07,1.013801e+07,2000.000000,2000.000000


In [23]:
Canceled_Df.head(10)

,Order Number,Line Number,Order Date,Ship To Address No,Item Code,Quantity Ordered,Quantity Canceled
0,528703,1,2017-01-03,1857566.0,10135139,1,1
1,528705,1,2017-01-03,1857566.0,10135140,1,1
2,528706,2,2017-01-03,1857566.0,10135138,1,1
3,528707,1,2017-01-03,1857566.0,10135132,1,1
4,528708,1,2017-01-03,1857566.0,10135359,1,1
5,528710,1,2017-01-03,1857566.0,10135364,1,1
6,528711,1,2017-01-03,1857566.0,10135358,1,1
7,528712,1,2017-01-03,1857566.0,10133703,1,1
8,528713,1,2017-01-03,1857566.0,10135131,1,1
9,528714,1,2017-01-03,1857566.0,10135363,1,1


In [42]:
Canceled_Df.describe()

,Order Number,Line Number,Ship To Address No,Item Code,Quantity Ordered,Quantity Canceled
count,3662.000000,3662.000000,3.662000e+03,3.662000e+03,3662.000000,3662.000000
mean,537084.880393,4.562534,8.950797e+06,6.700760e+06,8.714637,8.714637
std,3498.398809,4.093325,1.600894e+07,4.424588e+06,85.915973,85.915973
min,528703.000000,1.000000,6.299800e+04,1.104410e+05,1.000000,1.000000
25%,534856.000000,1.000000,1.255616e+06,1.040774e+06,1.000000,1.000000
50%,537724.000000,3.000000,1.653099e+06,1.013370e+07,1.000000,1.000000
75%,539391.750000,6.000000,2.246049e+06,1.013536e+07,3.000000,3.000000
max,543491.000000,27.000000,4.382536e+07,1.013801e+07,4500.000000,4500.000000


In [22]:
Sales_Df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28315 entries, 0 to 28314
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Order No            28315 non-null  int64  
 1   Order Date          28315 non-null  object 
 2   Line No             28315 non-null  int64  
 3   Ship To Address No  28315 non-null  float64
 4   Item Code           28315 non-null  int64  
 5   Order Qty           28315 non-null  float64
 6   Quantity Shipped    28315 non-null  float64
dtypes: float64(3), int64(3), object(1)
memory usage: 1.5+ MB


In [21]:
Canceled_Df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3662 entries, 0 to 3661
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Order Number        3662 non-null   int64  
 1   Line Number         3662 non-null   int64  
 2   Order Date          3662 non-null   object 
 3   Ship To Address No  3662 non-null   float64
 4   Item Code           3662 non-null   int64  
 5   Quantity Ordered    3662 non-null   int64  
 6   Quantity Canceled   3662 non-null   int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 200.4+ KB


In [44]:
Sales_Df[['Order Date','Quantity Shipped']].groupby('Order Date').sum().describe()


,Quantity Shipped
count,51.000000
mean,3592.843137
std,3272.035161
min,9.000000
25%,1280.500000
50%,2959.000000
75%,4350.000000
max,16656.000000


In [33]:
Canceled_Df[['Order Date','Quantity Ordered']].groupby('Order Date').sum()

,Quantity Ordered
Order Date,
2017-01-03,10
2017-01-04,1432
2017-01-05,41
2017-01-06,441
2017-01-07,35
2017-01-09,171
2017-01-10,1761
2017-01-11,47
2017-01-12,50
